In [1]:
#安装相关依赖
%pip install modelscope peft transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
#数据集下载
!modelscope download --dataset himzhzx/muice-dataset-train.catgirl --local_dir ./data

Processing 4 items:   0% 0.00/4.00 [00:00<?, ?it/s]









Processing 4 items:  25% 1.00/4.00 [00:01<00:04, 1.54s/it]
Processing 4 items:  50% 2.00/4.00 [00:01<00:01, 1.33it/s]

Processing 4 items:  75% 3.00/4.00 [00:02<00:00, 1.53it/s]


Processing 4 items: 100% 4.00/4.00 [00:03<00:00, 1.14it/s]


In [3]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",cache_dir='./models')

2025-05-16 13:42:53,314 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-05-16 13:46:41,327 - modelscope - INFO - Download model 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' successfully.
2025-05-16 13:46:41,328 - modelscope - INFO - Creating symbolic link [./models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B].


In [4]:
#相关库进行导入
#使用 lora 进行微调
import json
import torch
from torch.utils.data import Dataset, DataLoader
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

In [5]:
#导入模型
model = AutoModelForCausalLM.from_pretrained("./models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenizer = AutoTokenizer.from_pretrained("./models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
#使用 tokenizer 对句子进行分词，并转换为索引
tokenizer.encode("雪雪，你为什么叫沐雪")

[151646, 100167, 100167, 3837, 56568, 100678, 99882, 103303, 100167]

In [14]:
#数据集处理
class Mydata(Dataset):
  def __init__(self, tokenizer, max_len = 255):
    with open("./data/muice-dataset-train.catgirl.json", 'r', encoding='utf-8') as f:
      self.json_data = json.load(f)
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.json_data)

  def __getitem__(self, index):
    data = self.json_data[index]
    question = data['instruction']
    answener = data['output']
    q = self.tokenizer.apply_chat_template(
        [{'role':'user','content':question}],
        tokenize = False,
        add_generation_prompt = True
    )
    q_input_ids = self.tokenizer.encode(q)
    a_input_ids = self.tokenizer.encode(answener)
    input_ids = q_input_ids + a_input_ids
    attention_mask = [1]*len(input_ids)
    labels = [-100]*len(q_input_ids) + a_input_ids

    if len(input_ids) > self.max_len:
      input_ids = input_ids[:self.max_len]
      attention_mask = attention_mask[:self.max_len]
      labels = labels[:self.max_len]
    else:
      padding_len = self.max_len - len(input_ids)
      input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_len
      attention_mask = attention_mask + [0]*padding_len
      labels = labels + [-100]*padding_len

    return {
        'input_ids':torch.tensor(input_ids, dtype=torch.long),
        'attention_mask':torch.tensor(attention_mask, dtype=torch.long),
        'labels':torch.tensor(labels, dtype=torch.long)
    }


dataset = Mydata(tokenizer)

# dataset[0]

train_data = DataLoader(dataset, batch_size=32, shuffle=True)

for i in train_data:
  print(i)
  break

{'input_ids': tensor([[151646, 151646, 151644,  ..., 151643, 151643, 151643],
        [151646, 151646, 151644,  ..., 151643, 151643, 151643],
        [151646, 151646, 151644,  ..., 151643, 151643, 151643],
        ...,
        [151646, 151646, 151644,  ..., 151643, 151643, 151643],
        [151646, 151646, 151644,  ..., 151643, 151643, 151643],
        [151646, 151646, 151644,  ..., 151643, 151643, 151643]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]])}


In [8]:
#配置 lora 参数
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj','v_proj','k_proj', 'o_proj']
)

In [9]:
#将 lora 与原始模型 合并在一起
lora_model = get_peft_model(model, lora_config)

In [10]:
#打印模型总参数，以及可训练参数和占比
lora_model.print_trainable_parameters()

trainable params: 4,358,144 || all params: 1,781,446,144 || trainable%: 0.2446


In [11]:
#配置训练参数

output_dir = './lora/train'

train_arg = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    learning_rate=2e-5,
    save_total_limit=2,
    fp16=False,
    report_to="none"
)

In [15]:
#创建Trainer
trainer = Trainer(
    model=lora_model,
    args=train_arg,
    train_dataset=dataset,
    processing_class=tokenizer
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
#开始训练
trainer.train()

Step,Training Loss
10,7.343500
20,7.088400
30,6.795900
40,6.437900
50,6.265600
60,6.029000
70,5.920800
80,5.636200
90,5.577500
100,5.486200


TrainOutput(global_step=120, training_loss=6.124208005269368, metrics={'train_runtime': 1620.8964, 'train_samples_per_second': 2.384, 'train_steps_per_second': 0.074, 'total_flos': 8981544805908480.0, 'train_loss': 6.124208005269368, 'epoch': 2.9440993788819876})

In [17]:
#将 lora 权重合并到 基础模型上
merge_model = lora_model.merge_and_unload() #合并
merge_model.save_pretrained("./merge_model")  #保存
tokenizer.save_pretrained("./merge_model")  #保存

('./merge_model/tokenizer_config.json',
 './merge_model/special_tokens_map.json',
 './merge_model/tokenizer.json')

In [28]:
#模型测试
question = "你身份是什么？"
q = tokenizer.apply_chat_template(
        [{'role':'user','content':question}],
        tokenize = False,
        add_generation_prompt = True
    )

In [29]:
#编码
inputs = tokenizer(q, return_tensors="pt").to("cuda")
inputs

{'input_ids': tensor([[151646, 151646, 151644,  56568, 101294, 102021,  11319, 151645, 151648,
            198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [30]:
#模型推理
lora_model.config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

with torch.no_grad():
  outputs = lora_model.generate(
      **inputs,
      max_new_tokens=200,
      temperature=0.7,
      do_sample=True
  )

output_text = tokenizer.decode(outputs[0])
output_text

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


'<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜><｜User｜>你身份是什么？<｜Assistant｜><think>\n我是由深度求索公司独立开发的DeepSeek-R1。关于不同模型的差异，建议您参考官方信息或亲自体验。我会继续专注于以诚实专业的态度为您提供最好的帮助。\n</think>\n\n我是由深度求索公司独立开发的DeepSeek-R1。关于不同模型的差异，建议您参考官方信息或亲自体验。我会继续专注于以诚实专业的态度为您提供最好的帮助。<｜end▁of▁sentence｜>'